# Explainability Framework - VQC



Below is the code used for analysing the different components of the VQC through various explainability techniques.

In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports

In [15]:
# imports
from matplotlib import pyplot as plt
import numpy as np

import os

# import data class
from utilities.dataset_utils import DiabetesData

# explainability imports
# feature map SHAP
from shap import KernelExplainer
from shap import Explanation
from shap.plots import waterfall
from qiskit.quantum_info import state_fidelity

# qiskit imports
# feature map
from qiskit.circuit.library import zz_feature_map

# simulator
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler

In [16]:
# path to diabetes.csv
path = os.path.join(os.getcwd(), '..', 'utilities', 'diabetes.csv')
# load dataset class
dataset = DiabetesData(path)

In [17]:
# list of feature names
feature_names = [
    "Pregnancies",
    "Glucose",
    "BloodPressure",
    "SkinThickness",
    "Insulin",
    "BMI",
    "DiabetesPedigreeFunction",
    "Age"
]

In [ ]:
# get data
X_train, X_test, y_train, y_test = dataset.preprocess_data_ranged()

In [19]:
# setup backend simulator
backend = AerSimulator()
backend.set_options(max_parallel_threads=os.cpu_count(), method='statevector')

In [20]:
# sampler
sampler = Sampler.from_backend(backend)

### Feature Map Explainability

To examine feature map explainability, the first method used was a SHAP based Feature-State contribution mapping. It analyses
 - Encoding Importance
   - The magnitude of the shap value indicates which features are most importance for encoding into a quantum state
 - Transformation to Quantum States
   - Positive SHAP values indicate that features cluster together, meaning features have similar quantum states and therefor have less distinction
   - Negative SHAP values would indicate the states are further apart in the Hilbert space, enhancing distinguishability between different classes

In [21]:
# generate feature map used
feature_map = zz_feature_map(feature_dimension=dataset.get_num_features(), reps=4, entanglement='full')

In [22]:
# function to get quantum state
def get_quantum_state(x):
    """
    Gets quantum state of a given data point
    """
    # copy circuit
    qc = feature_map.copy()
    
    # param dict
    params = dict(zip(qc.parameters, x))
    
    # assign params to circuit
    qc = qc.assign_parameters(params)
    
    # save statevector
    qc.save_statevector()
    
    # Run on statevector simulator
    result = backend.run(qc).result()
    return result.get_statevector()

In [23]:
# function to calculate fidelity between two data points
def state_similarity(x1, x2):
    """
    Measures similarity between two states
    """
    # get quantum states for 2 data points
    state1 = get_quantum_state(x1)
    state2 = get_quantum_state(x2)
    
    # calculate state fidelity
    fidelity = np.abs(state_fidelity(state1, state2))**2
    return fidelity

In [24]:
def quantum_feature_map_shap(feature_map, x_sample, background_data, feature_names=None):
    """
    Calculate SHAP values specifically for a quantum feature map to explain how 
    different features contribute to the quantum state representation
    
    Args:
        feature_map: Feature map circuit from zz_feature_map
        x_sample: Data point to explain
        background_data: Background dataset for reference
        feature_names: Names of the features
    """
    
    explainer = KernelExplainer(
        model=lambda x: np.array([state_similarity(x_i, x_sample) for x_i in x]), 
        data=background_data
    )
    
    # calculate shap values from explainer
    shap_values = explainer.shap_values(x_sample.reshape(1, -1))
    
    # Create a consistent visualization regardless of SHAP version
    plt.figure(figsize=(10, 6))
    
    explanation = Explanation(
        values=shap_values[0],
        base_values=explainer.expected_value,
        data=x_sample,
        feature_names=feature_names
    )
    waterfall(explanation, show=False)
    
    plt.title('Feature Importance for Quantum Feature Map (Diabetes Dataset)')
    plt.tight_layout()
    plt.show()
    
    return shap_values

In [25]:
# Select a sample to explain
sample_idx = 0  # You can change this to any index
x_sample = X_test[sample_idx]
true_label = y_test[sample_idx]

# Use a subset of training data as background
background_data = X_train[:100]  # Using first 100 samples as background

# calculate SHAP values for the feature map
shap_values = quantum_feature_map_shap(
    feature_map=feature_map,
    x_sample=x_sample,
    background_data=background_data,
    feature_names=feature_names
)

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 